Gabriel Marcelino
September 2024
Artificial Neural Network (ANN)

## Import dependencies and load data

In [197]:
import csv
import random
import numpy as np
from keras.models import Sequential
from keras.layers import Dense
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

pool = []
training_data = []
# create pool with players from 2019-2022
with open('all_seasons.csv', mode = 'r') as file:
    csvFile = csv.reader(file)
    # ignore first line
    next(csvFile)
    for lines in csvFile:
        year = int(lines[21][:4])
        if 2018 < year < 2023 and len(pool) < 100 and lines[1] not in pool:
            pool.append(lines)
        elif len(training_data) < 5000:
            training_data.append(lines)


[['10631', 'Jared Dudley', 'LAL', '34.0', '198.12', '107.501304', 'Boston College', 'USA', '2007', '1', '22', '45', '1.5', '1.2', '0.6', '3.3', '0.017', '0.11900000000000001', '0.081', '0.5579999999999999', '0.10400000000000001', '2019-20'], ['10632', 'Jabari Parker', 'SAC', '25.0', '203.2', '111.13004', 'Duke', 'USA', '2014', '1', '2', '38', '14.0', '5.6', '1.8', '-4.9', '0.065', '0.17', '0.233', '0.5710000000000001', '0.11', '2019-20'], ['10633', 'James Johnson', 'MIN', '33.0', '200.66', '108.86208', 'Wake Forest', 'USA', '2009', '1', '16', '32', '8.4', '3.7', '2.3', '-4.6', '0.038', '0.145', '0.18600000000000003', '0.5720000000000001', '0.177', '2019-20'], ['10634', 'James Harden', 'HOU', '30.0', '195.58', '99.79024', 'Arizona State', 'USA', '2009', '1', '3', '68', '34.3', '6.6', '7.5', '5.8', '0.026000000000000002', '0.139', '0.35600000000000004', '0.626', '0.366', '2019-20'], ['10635', 'James Ennis III', 'ORL', '29.0', '198.12', '97.52228000000001', 'California State-Long Beach', 

## Optimal Team : Considerations
For my optimal team, I will aim for:
- 2 or more players in top 20% of shooting percentage.
- Player in the top 5% out of the 100 for rebounds.
- Player with a Defensive Rebound percentage bigger than 0.2
- 3 or more players in top 20% of best net rating
- 2 or more players with better than average assists

## Model

In [198]:
def extract_features(pool):
    features_list = []
    for player in pool:
        # extract relevant features based on considerations above
        features = {
            'name': player[1],
            'ts_pct': player[19],
            'reb': player[13],
            'dreb_pct': player[17],
            'rating': player[15],
            'ast': player[14]
        }
        features_list.append(features)
    return features_list


## Simulate Training Data to train model

In [199]:


def simulate_data(sample, num_iter=10000):
    X = []
    y = []
    # calculate average assists
    assists = np.array([float(player[14]) for player in sample])
    average_assists = np.mean(assists)
    # calculate top 20% of shooting percentage
    top_20_ts = np.percentile([float(player[19]) for player in sample], 80)
    # calculate top 10% of rebound
    top_10_reb = np.percentile([float(player[13]) for player in sample], 90)
    # calculate top 20% net rating
    top_20_rating = np.percentile([float(player[15]) for player in sample], 80)

    for i in range(num_iter):
        # select 5 random players from list
        selected_players = random.sample(sample, 5)
        features = extract_features(selected_players)
        X.append(features)
        """
        - 2 or more players in top 20% of shooting percentage.
        - Player in the top 10% for rebounds.
        - Player with a Defensive Rebound percentage bigger than 0.2
        - 3 or more players in top 20% of best net rating
        - 2 or more players with better than average assists
        """
        label = 0
        # check if there are 2 players with better than average assists
        players_ast = [player for player in features if float(player['ast']) > average_assists]
        if len(players_ast) >= 2:
            # check if 2 or more players in top 20% of shooting percentage
            players_ts = [player for player in features if float(player['ts_pct']) > top_20_ts]
            if len(players_ts) >= 2:
                # check if any player is in the top 10% for rebounds
                players_reb = [player for player in features if float(player['reb']) > 1]
                if len(players_reb) >=1:
                    # check if any player on team has dreb pct > 0.2
                    players_dreb = [player for player in features if float(player['dreb_pct'])>0.2]
                    if len(players_dreb) >=1:
                        # check if 3 or more players in top 20% of net rating
                        players_rating = [player for player in features if float(player['rating']) > top_20_rating]
                        if len(players_rating) >= 3:
                            # Optimal Team Found
                            label = 1
        y.append(label)                   
    X = np.array(X)
    y = np.array(y)
    print(f"Shape of X: {X.shape}")
    print(f"Shape of y: {y.shape}")
    # TO-DO
    # Only 20 ish positives out of the sample with 10000
    # Maybe change rules ?
    count_ones = np.sum(y == 1)
    print(f"Number of 1's in y: {count_ones}")

    return X, y




## Train Model
Now that we have the training data, we can train the model.

In [200]:
X_train, y_train = simulate_data(training_data)

# Extract numerical features from dictionaries
X_train = np.array([
    [
        float(player['ts_pct']),
        float(player['reb']),
        float(player['dreb_pct']),
        float(player['rating']),
        float(player['ast'])
    ]
    for team in X_train
    for player in team
]).reshape(len(X_train), -1)

# Build the model
model = Sequential()
model.add(Dense(64, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(1, activation='sigmoid'))

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Train the model
model.fit(X_train, y_train, epochs=10, batch_size=32)
# TO-DO: SPLIT DATA FOR TESTING?

Shape of X: (10000, 5)
Shape of y: (10000,)
Number of 1's in y: 130
Epoch 1/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 454us/step - accuracy: 0.9603 - loss: 0.1494
Epoch 2/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 423us/step - accuracy: 0.9826 - loss: 0.0733
Epoch 3/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 424us/step - accuracy: 0.9873 - loss: 0.0561
Epoch 4/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 431us/step - accuracy: 0.9883 - loss: 0.0520
Epoch 5/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 436us/step - accuracy: 0.9872 - loss: 0.0492
Epoch 6/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 459us/step - accuracy: 0.9866 - loss: 0.0493
Epoch 7/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 481us/step - accuracy: 0.9866 - loss: 0.0447
Epoch 8/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 451us/step - accuracy: 0.9870 - loss: 0.0425
Epoch 9/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 470us/step - accuracy: 0.9891 - loss: 0.0371
Epoch 10/10
313/313 ━━━━━━━━━━━━━━━━━━━━ 0s 494us/step - accuracy: 0.9877 - loss: 0.0380


TO DO:

Explain your architecture and how the basketball player characteristics are used as inputs:

Data Splitting for Testing:
Split the data and try to predict the testing data



Only 100 ish positives out of the sample with 10000 when simulating data
Maybe change requirements for optimal team so one can be more commom ?

Double check algorithm and outputs

Expand documentation

Interpret the output of your MLP in the context of selecting an optimal basketball team:

